In [1]:
import pandas as pd
import requests
import glob
import time
import cv2
import os
import dlib
import numpy as np
from matplotlib import pyplot as plt
import traceback
from typing import List

In [2]:
task = pd.read_csv("video_tasks_50.csv")
task

,task_id,task,matirial
0,0,A woman rock-climbs at an indoor climbing faci...,[]
1,1,A horse stands to the right of a grazing sheep...,[]
2,2,宇航员在月球表面上打高尔夫球,[]
3,3,公园长椅上有一只橘猫在伸爪抓蝴蝶，同时左侧有小孩在追逐金色的气球,[]
4,4,埃及金字塔顶上有两个小孩在放风筝,[]
5,5,一个穿格子衫的程序员坐在时代广场旁的凳子上写代码,[]
6,6,一个10岁的小男孩穿着橙色的衣服和绿色的裤子，用右手摸一下眉毛，背后是埃菲尔铁塔，全身的视角,[]
7,7,一个坐在玉米地边上的老爷爷，在喝完水之后，右手摸一下额头，半身的视角,[]
8,8,一位中年男子戴着黑色鸭舌帽，抱着一只棕色猫，用一根手指戳了一下左脸，半蹲姿，全身视角,[]
9,9,一个小男孩穿红色T恤和牛仔短裤，举了左手后又放下，旁边是沙滩球和太阳伞，半身视角,[]


In [3]:
import urllib.request

def download_video(url, save_path):
    try:
        urllib.request.urlretrieve(url, save_path)
        print(f"Video successfully downloaded to {save_path}")
    except Exception as e:
        print(f"Error downloading video: {e}")

# 填入你的key
# https://bailian.console.aliyun.com/?tab=model#/api-key
DASHSCOPE_API_KEY = "sk-6bXXXX"


def convert_text_prompt_to_video(prompt: str):
    """
    https://bailian.console.aliyun.com/?spm=5176.29619931.0.0.74cd521ctl89Fw&tab=api#/api/?type=model&url=https%3A%2F%2Fhelp.aliyun.com%2Fdocument_detail%2F2865250.html&renderType=iframe
    """
    
    url4generate = "https://dashscope.aliyuncs.com/api/v1/services/aigc/video-generation/video-synthesis"
    headers = {
        "X-DashScope-Async": "enable",
        "Authorization": f"Bearer {DASHSCOPE_API_KEY}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "wanx2.1-t2v-turbo",
        "input": {
            "prompt": prompt,
        },
        "parameters": {
            "size": "1280*720"
        }
    }
    
    task_id = None
    for _ in range(3):
        try:
            response = requests.post(url4generate, headers=headers, json=data)
            task_id = response.json()["output"]["task_id"]
            break
        except:
            traceback.print_exc()
            pass

    print(task_id)
    url4result = f"https://dashscope.aliyuncs.com/api/v1/tasks/{task_id}"
    headers = {
        "Authorization": f"Bearer {DASHSCOPE_API_KEY}"
    }
    for _ in range(60):
        try:
            response = requests.get(url4result, headers=headers)
            if response.json()["output"]["task_status"] == "SUCCEEDED":
                return response.json()["output"]["video_url"]
            else:
               time.sleep(5)
        except:
            traceback.print_exc()
            pass

    return None

def convert_text_prompt_and_ref_to_video(prompt: str, function_str:str, video_url:str=None, img_url_list: List[str]=None):
    """
    https://bailian.console.aliyun.com/?spm=5176.29619931.0.0.74cd521ctl89Fw&tab=api#/api/?type=model&url=https%3A%2F%2Fhelp.aliyun.com%2Fdocument_detail%2F2865250.html&renderType=iframe
    """
    
    url4generate = "https://dashscope.aliyuncs.com/api/v1/services/aigc/video-generation/video-synthesis"
    headers = {
        "X-DashScope-Async": "enable",
        "Authorization": f"Bearer {DASHSCOPE_API_KEY}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "wanx2.1-vace-plus",
        "input": {
            "prompt": prompt,
            "function": function_str,
        },
        "parameters": {
            "size": "1280*720"
        }
    }

    if video_url:
        data["input"]["video_url"] = video_url

    if img_url_list:
        data["input"]["ref_images_url"] = img_url_list
        data["parameters"]["obj_or_bg"] = ["obj"] * len(img_url_list)

    print(data)
    task_id = None
    for _ in range(3):
        try:
            response = requests.post(url4generate, headers=headers, json=data)
            task_id = response.json()["output"]["task_id"]
            break
        except:
            traceback.print_exc()
            pass

    print(task_id)
    url4result = f"https://dashscope.aliyuncs.com/api/v1/tasks/{task_id}"
    headers = {
        "Authorization": f"Bearer {DASHSCOPE_API_KEY}"
    }
    for _ in range(60):
        try:
            response = requests.get(url4result, headers=headers)
            if response.json()["output"]["task_status"] == "SUCCEEDED":
                return response.json()["output"]["video_url"]
            else:
                print(response.json())
                time.sleep(5)
        except:
            traceback.print_exc()
            pass

    return None

In [4]:
!mkdir generate_result

mkdir: generate_result: File exists


In [5]:
for row in task.iterrows():
    if os.path.exists("./generate_result/" + str(row[1].task_id) + ".mp4"):
        continue

    if len(eval(row[1].matirial)) == 0:
        video_url = convert_text_prompt_to_video(row[1].task)
    else:
        matirial = eval(row[1].matirial)
        matirial_video = [x for x in matirial if x.endswith(".mp4")]
        if len(matirial_video) != 0:

            video_url = convert_text_prompt_and_ref_to_video(row[1].task, "video_repainting", 
                                         video_url="http://mirror.coggle.club/tianchi/532391/" + matirial_video[0])
            if not video_url:
                video_url = 'http://mirror.coggle.club/tianchi/532391/30.mp4'

        else:
            video_url = convert_text_prompt_and_ref_to_video(row[1].task, "image_reference", 
                                         img_url_list=["http://mirror.coggle.club/tianchi/532391/" + x for x in matirial])

        
    download_video(video_url, "./generate_result/" + str(row[1].task_id) + ".mp4")

Video successfully downloaded to ./generate_result/30.mp4
Video successfully downloaded to ./generate_result/31.mp4
Video successfully downloaded to ./generate_result/32.mp4
Video successfully downloaded to ./generate_result/33.mp4
Video successfully downloaded to ./generate_result/34.mp4
Video successfully downloaded to ./generate_result/35.mp4
Video successfully downloaded to ./generate_result/36.mp4
Video successfully downloaded to ./generate_result/37.mp4
Video successfully downloaded to ./generate_result/38.mp4
Video successfully downloaded to ./generate_result/39.mp4
Video successfully downloaded to ./generate_result/40.mp4
Video successfully downloaded to ./generate_result/41.mp4
Video successfully downloaded to ./generate_result/42.mp4
Video successfully downloaded to ./generate_result/43.mp4
Video successfully downloaded to ./generate_result/44.mp4
Video successfully downloaded to ./generate_result/45.mp4
Video successfully downloaded to ./generate_result/46.mp4
Video successf

In [7]:
!zip -r generate_result.zip generate_result

  adding: generate_result/ (stored 0%)
  adding: generate_result/7.mp4 (deflated 0%)
 (deflated 0%)rate_result/41.mp4
  adding: generate_result/40.mp4 (deflated 0%)
  adding: generate_result/6.mp4 (deflated 0%)
  adding: generate_result/42.mp4 (deflated 0%)
 (deflated 0%)rate_result/4.mp4
 (deflated 0%)rate_result/5.mp4
  adding: generate_result/43.mp4 (deflated 0%)
  adding: generate_result/47.mp4 (deflated 0%)
 (deflated 0%)rate_result/1.mp4
  adding: generate_result/0.mp4 (deflated 0%)
  adding: generate_result/46.mp4 (deflated 0%)
 (deflated 0%)rate_result/2.mp4
  adding: generate_result/44.mp4 (deflated 0%)
  adding: generate_result/45.mp4 (deflated 0%)
 (deflated 0%)rate_result/3.mp4
  adding: generate_result/36.mp4 (deflated 0%)
 (deflated 0%)rate_result/22.mp4
 (deflated 0%)rate_result/23.mp4
  adding: generate_result/37.mp4 (deflated 0%)
 (deflated 0%)rate_result/21.mp4
 (deflated 0%)rate_result/35.mp4
  adding: generate_result/34.mp4 (deflated 0%)
 (deflated 0%)rate_result/20

In [8]:
task

,task_id,task,matirial
0,0,A woman rock-climbs at an indoor climbing faci...,[]
1,1,A horse stands to the right of a grazing sheep...,[]
2,2,宇航员在月球表面上打高尔夫球,[]
3,3,公园长椅上有一只橘猫在伸爪抓蝴蝶，同时左侧有小孩在追逐金色的气球,[]
4,4,埃及金字塔顶上有两个小孩在放风筝,[]
5,5,一个穿格子衫的程序员坐在时代广场旁的凳子上写代码,[]
6,6,一个10岁的小男孩穿着橙色的衣服和绿色的裤子，用右手摸一下眉毛，背后是埃菲尔铁塔，全身的视角,[]
7,7,一个坐在玉米地边上的老爷爷，在喝完水之后，右手摸一下额头，半身的视角,[]
8,8,一位中年男子戴着黑色鸭舌帽，抱着一只棕色猫，用一根手指戳了一下左脸，半蹲姿，全身视角,[]
9,9,一个小男孩穿红色T恤和牛仔短裤，举了左手后又放下，旁边是沙滩球和太阳伞，半身视角,[]
